<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT19/blob/jledeus/jledeus_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative methods**
**Johan Ledéus**

# **Introduction**

Homework 3 for DD2363 Methods in Scientific Computing


# Methods

## Mandatory assignments

###1. Function: Jacobi iteration for $Ax=b$

####Input: matrix $A$, vector $b$
####Output: vector $x$
####Test: convergence of residual $|| Ax-b ||$, $|| x-y ||$ for manufactured/exact solution $y$ 

#### Solution (Chapter 7 in course litterature)
The idea with iterative methods is that we don't try to find the exact soultion for $Ax=b$ but rather the approximation.
The matrix $A$ can be splitted into the diagonal matrix $A_1 = D$ and $A_2 = A-D$

$$A = A_1 + A_2 = D + A - D = A$$

Given this the jacobi iteration is defined as:

$$x_i^{k+1} = a_{ii}^{-1}(b_i-\sum_{j \neq i}a_{ij}x_j^{(k)})$$

The stopping criterion is defined as:
$$\frac{||r^{(k)}||}{||b||} < TOL, r^{(k)} = b-Ax^{(k)}$$


In [0]:
import numpy as np

def jacobi_iteration(A,b):
  x = [1 for _ in b] # x_o = [1,1,..,1]
  while True:
    x_new = x
    for i in range(len(x)):
      x_new[i] = (1/A[i][i])*(b[i] - sum([A[i][j]*x[j] for j in range(len(A[0])) if j != i]))
    x = x_new
    if np.linalg.norm(b-A.dot(x))/np.linalg.norm(b) < 0.000000000001:
      break
  return x

###2. Function: Gauss-Seidel iteration for $Ax=b$

####Input: matrix $A$, vector $b$
####Output: vector $x$
####Test: convergence of residual $|| Ax-b ||$, $|| x-y ||$ for manufactured/exact solution $y$

#### Solution
First we split the matrix $A$ into the lower triangular matrix $L$ and the strictly upper triangular matrix $U = A - L$. With this we can construct the iterative method with:

$$Ax = B \iff (L + U)x = b \iff x^{k+1}=L^{-1}(b - Ux^{k})  $$

With the knowledge that $L$ is an lower triangular matrix we can use the forwards substitution and hence acquire:

$$x_i^{(k+1)} = a_{ii}^{-1}(b - \sum_{j<i}a_{ij}x_j^{k+1} - \sum_{j>i}a_{ij}x_j^{(k)})$$

In [0]:
def gauss_seidel(A,b):
  x = [1 for _ in b] # x_o = [1,1,..,1]
  while True:
    x_new = x
    for i in range(len(x)):
      r_sum = sum([A[i][j]*x_new[j] for j in range(len(A[0])) if j < i])
      l_sum = sum([A[i][j]*x[j] for j in range(len(A[0])) if j > i])
      x_new[i] = (1/A[i][i])*(b[i] - l_sum - r_sum)
    x = x_new
    if np.linalg.norm(b-A.dot(x))/np.linalg.norm(b) < 0.000000000001:
      break
  return x

###3. Function: Newton's method for scalar nonlinear equation $f(x)=0$

####Input: scalar function $f(x)$
####Output: real number $x$
####Test: convergence of residual $|f(x)|$, $|x-y|$ for manufactured/exact solution $y$

####Solution (Algorithm 13 in course litterature)
The purpose of Newton's method is to find roots for equations.

$x^{(0)} \in ℝ $

$while |f(x^{(k)})| \geq TOL $

$ x^{(k+1)} = x^{(k)} - \frac{f(x^{(k)})}{f'(x^{(k)})} $

$end$

In [0]:
def newtons_method(f, h=0.0001):
  x = 0
  f_prim = lambda f, x, h : (f(x+h)-f(x)) / h
  while np.abs(f(x)) > 0.000001:
    x = x - f(x)/f_prim(f,x, h)
  return x  

# Results

### 1. Jacobi Iteration
As we can see below the results for the jacobi iteration is accurate given the algorithm. Depending on the tolerance $TOL$ we might get a different result.

In [107]:
# Test for assignment 1
A = np.array([(10,-1,2,0),(-1,11,-1,3),(2,-1,10,-1),(0,3,-1,8)])
b = np.array([6,25,-11,15])
x = jacobi_iteration(A,b)
y = np.array([1, 2, -1, 1]) # Exact solution of the system

assert np.linalg.norm(A.dot(x) - b) < 0.0000000001
assert np.linalg.norm(x-y) < 0.0000000001
print("Pass")

Pass


###2. Gauss-Seidel Iteration
Similar to the previous assignment the results are promising when it comes to approximating the solution to the equation $Ax=b$

In [108]:
# Test code for asignment 2
A = np.array([(10,-1,2,0),(-1,11,-1,3),(2,-1,10,-1),(0,3,-1,8)])
b = np.array([6,25,-11,15])
x = gauss_seidel(A,b)
y = np.array([1, 2, -1, 1]) # Exact solution of the system

assert np.linalg.norm(A.dot(x) - b) < 0.0000000001
assert np.linalg.norm(x-y) < 0.0000000001
print("Pass")

Pass


###3. Newton's method
The results for this metod depends on the initial guess for $x^{(0)}$ and the tolerance we use for derivation and termination criterion. Further improvements would be to include more arguments in order to generate more solutions.

In [109]:
# Test code for assignment 3
f = lambda x : x**2 - 1395
x = newtons_method(f)
assert np.abs(f(x)) < 0.000001
assert np.abs(x - np.sqrt(1395)) < 0.000001

f = lambda x : x**3 - 10*x + 2
x = newtons_method(f,0.000001)

assert np.abs(f(x)) < 0.000001
assert np.abs(x - 0.20081) < 0.000001

print("Pass")

Pass


# Discussion

The mandatory assignments was pretty straight forward but the extra which I only attempted GMRES was somewhat confusing to implement. I had some problems to implement the combination of Arnoldi with GMRES and due to lack of time I did not finish any extra assignments. As mentioned in lecture I think it's interesting to discuss the abilities when it comes to parallelism and dependencies of data.